# STCH-qPMHI Tutorial

This notebook demonstrates the **STCH-qPMHI two-stage batch selection framework** for multi-objective Bayesian optimization.

## Overview

STCH-qPMHI combines:
1. **Stage 1 (STCH Candidate Generation)**: Efficiently generates diverse candidate pool using STCH scalarization with multiple weight vectors
2. **Stage 2 (qPMHI Batch Selection)**: Selects optimal batch by ranking candidates using Probability of Maximum Hypervolume Improvement

This approach leverages gradient-based STCH optimization for exploration, combined with hypervolume-optimal batch selection via qPMHI.

**Note**: This framework adapts the qPMHI algorithm from Muthyala et al. (2025) to work with STCH scalarization for continuous optimization problems.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils.multi_objective.hypervolume import Hypervolume
from gpytorch.mlls import ExactMarginalLogLikelihood

# Import STCH-qPMHI (when implementation is available)
# from stch_botorch import optimize_stch_qpmhi, qPMHI
# from stch_botorch.integration import STCHqPMHIAcquisition, STCHCandidateGenerator
from stch_botorch import smooth_chebyshev

torch.manual_seed(42)
np.random.seed(42)

print("STCH-qPMHI Tutorial")
print("=" * 50)

## Setup: Problem and Initial Data

We'll use a simple 2-objective problem for demonstration. In practice, STCH-qPMHI works with any continuous multi-objective optimization problem.

In [ ]:
# Simple 2D input, 2-objective problem for demonstration
def simple_multi_obj(X):
    """Simple 2-objective function for demonstration."""
    obj1 = -((X[:, 0] - 0.5) ** 2 + (X[:, 1] - 0.5) ** 2)  # Maximize (negate for minimization)
    obj2 = -((X[:, 0] - 0.3) ** 2 + (X[:, 1] - 0.7) ** 2)
    return torch.stack([obj1, obj2], dim=-1)

# Initial data
n_init = 8
train_X = torch.rand(n_init, 2, dtype=torch.double)
train_Y = simple_multi_obj(train_X)

bounds = torch.stack([torch.zeros(2, dtype=torch.double), torch.ones(2, dtype=torch.double)])

print(f"Initial data: {train_X.shape[0]} points")
print(f"Objectives: {train_Y.shape[1]}")
print(f"Bounds: {bounds}")

## Helper Functions

In [ ]:
def compute_pareto(Y):
    """Compute Pareto front (maximization)."""
    if Y.shape[0] == 0:
        return Y
    
    n, m = Y.shape
    is_pareto = torch.ones(n, dtype=torch.bool, device=Y.device)
    
    for i in range(n):
        if not is_pareto[i]:
            continue
        y_i = Y[i:i+1]
        
        for j in range(i+1, n):
            if not is_pareto[j]:
                continue
            y_j = Y[j:j+1]
            
            if (y_j >= y_i).all() and (y_j > y_i).any():
                is_pareto[i] = False
                break
            if (y_i >= y_j).all() and (y_i > y_j).any():
                is_pareto[j] = False
    
    return Y[is_pareto]

## Stage 1: STCH Candidate Generation (Conceptual)

In the full implementation, STCH-qPMHI uses `STCHCandidateGenerator` to:
1. Generate multiple weight vectors (e.g., via Sobol sampling on simplex)
2. For each weight, optimize STCH scalarization using gradient-based optimization
3. Collect diverse candidates covering the Pareto front

Here we demonstrate the concept:

In [ ]:
# Fit GP model
model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)

print("Model fitted successfully")

# Compute Pareto front and reference point
pareto_Y = compute_pareto(train_Y)
ref_point = torch.tensor([-1.0, -1.0], dtype=torch.double)  # Reference point for hypervolume

print(f"Pareto front size: {pareto_Y.shape[0]}")
print(f"Reference point: {ref_point}")

## Stage 2: qPMHI Batch Selection (Conceptual)

The qPMHI algorithm (from Muthyala et al., 2025) scores each candidate by the probability that it achieves the maximum hypervolume improvement. The key insight is that qPMHI has an additive decomposition property, enabling exact batch selection via simple ranking.

**Note**: The full implementation will use the `optimize_stch_qpmhi` function or `STCHqPMHIAcquisition` class. See README.md for complete usage example.

In [ ]:
# Example: Using STCH-qPMHI (when implementation is available)
# batch = optimize_stch_qpmhi(
#     model=model,
#     bounds=bounds,
#     pareto_Y=pareto_Y,
#     ref_point=ref_point,
#     q=4,  # Batch size
#     num_candidates=100,  # Candidate pool size
#     stch_kwargs={"num_weights": 50, "num_restarts": 3},
# )

print("\nSTCH-qPMHI Framework:")
print("1. Stage 1: STCH generates diverse candidate pool")
print("2. Stage 2: qPMHI ranks candidates and selects top-q")
print("\nSee README.md for complete usage example.")

## Understanding the Two-Stage Framework

### Why STCH-qPMHI?

**STCH Stage 1 Advantages:**
- Gradient-based optimization (efficient)
- Theoretically proven Pareto coverage
- No generative model required
- Works with continuous optimization problems

**qPMHI Stage 2 Advantages:**
- Hypervolume-optimal batch selection
- Additive decomposition enables exact selection via ranking
- Scalable to large pools and batches
- From Muthyala et al. (2025) - properly cited

**Combined Benefits:**
- Pareto coverage guarantees (STCH) + Hypervolume optimality (qPMHI)
- Efficient exploration + Optimal batch selection
- Works for continuous problems without generative models

In [ ]:
# Visualize current Pareto front
fig, ax = plt.subplots(figsize=(8, 6))

ax.scatter(train_Y[:, 0], train_Y[:, 1], alpha=0.5, s=50, label="All points")
if pareto_Y.shape[0] > 0:
    # Sort for visualization
    sort_idx = torch.argsort(pareto_Y[:, 0])
    pareto_sorted = pareto_Y[sort_idx]
    ax.plot(pareto_sorted[:, 0], pareto_sorted[:, 1], "r-o", markersize=8, linewidth=2, label="Pareto front")

ax.set_xlabel("Objective 1", fontsize=12)
ax.set_ylabel("Objective 2", fontsize=12)
ax.set_title("Current Pareto Front", fontsize=14, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Comparison: STCH-qPMHI vs Other Approaches

| Approach | Stage 1 | Stage 2 | Best For |
|----------|---------|---------|----------|
| **STCH-qPMHI** (ours) | STCH scalarization | qPMHI ranking | Continuous MOBO, no generative model needed |
| **Generative qPMHI** (paper) | Generative model (VAE/GAN) | qPMHI ranking | Discrete/molecular design, with generative model |
| **qNEHVI** | Direct optimization | Hypervolume improvement | Small batches, exact hypervolume |
| **qParEGO** | Random weights | Scalarized optimization | Single-objective scalarization |

See README.md for detailed comparison and when to use each approach.

## Summary

This tutorial demonstrated the STCH-qPMHI two-stage framework:

1. **STCH Candidate Generation**: Uses gradient-based STCH scalarization with multiple weight vectors to generate diverse candidate pool
2. **qPMHI Batch Selection**: Uses probability of maximum hypervolume improvement to rank and select optimal batch

**Key Takeaways:**
- STCH-qPMHI combines efficient exploration (STCH) with optimal batch selection (qPMHI)
- Works for continuous optimization problems without requiring generative models
- Provides both Pareto coverage guarantees and hypervolume optimality
- qPMHI algorithm from Muthyala et al. (2025) - properly cited

For complete implementation and usage examples, see the README.md and the `stch_botorch.integration` module (when available).